In [60]:
import numpy as np
from collections import Counter

In [62]:
class node:
    def __init__(self,left_tree=None,right_tree=None,feature=None,threshold=None,value=None):
        self.left_tree=left_tree
        self.right_tree=right_tree
        self.feature=feature
        self.threshold=threshold
        self.value=value

    def is_leaf_node(self,node):
        return node.value is not None
            
    
    

In [64]:
class DecisionTreeClassifier:
    def __init__(self,max_depth=100,min_samples=2):
        self.max_depth=max_depth
        self.min_samples=min_samples
        self.root=None

    def entropy(self,y):
        labels,counts=np.unique(y,return_counts=True)
        probs=counts/len(y)
        return -np.sum(probs*np.log2(probs + 1e-15))

    def info_gain(self,y,left_idx,right_idx):
        root_ent=self.entropy(y)

        left_child=y[left_idx]
        right_child=y[right_idx]

        left_weight=len(left_idx)/len(y)
        right_weight=len(right_idx)/len(y)

        left_ent=self.entropy(left_child)
        right_ent=self.entropy(right_child)

        return (root_ent - (left_ent*left_weight + right_ent*right_weight))

    def best_split(self,X,y):
        best_feature=None
        gain=-1
        best_threshold=None
        best_left_idx=None
        best_right_idx=None

        for feature in range(X.shape[1]):
            thresholds=np.unique(X[:,feature])

            for t in thresholds:

                left_idx=np.where(X[:,feature]<=t)[0]
                right_idx=np.where(X[:,feature]>t)[0]

                if len(left_idx)==0 or len(right_idx)==0:
                    continue

                gain=self.info_gain(y,left_idx,right_idx)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = t
                    best_left_idx = left_idx
                    best_right_idx = right_idx

        return best_feature,best_threshold,best_left_idx,best_right_idx

    def  most_common_label(self,y):
        return counter(y).most_common(1)[0][0]
        

    def build_tree(self,X,y,depth=0):
        nsamples,nfeatures=X.shape
        labels=np.unique(y)

        # stopping criteria

        if (len(labels)==1 or depth>=self.max_depth or nsamples<self.min_samples):
            leaf_value=self.most_common_label(y)
            return node(value=leaf_value)

        feature,threshold,left_idx,right_idx=self.best_split(X,y)

        left_tree=self.build_tree(X[left_idx],y[left_idx],depth+1)
        right_tree=self.build_tree(X[right_idx],y[right_idx],depth+1)
        
        return node(left_tree,right_tree,feature,threshold)

    def fit(self,X,y):
        self.root=self.build_tree(X,y)

    def predict_sample(self,x,node):

        if is_leaf_node(node):
            return node.value

        if x[node.feature]<=node.threshold:
            return predict_sample(x,node.left_tree)
        else:
            return predict_sample(x,node.right_tree)

    def predict(self,X):
        return np.array([self.predict_sample(X,self.root) for x in X])



In [70]:
class RandommForestClassifier:
    def __init__(self,n_estimators=10,max_depth=100,min_samples=2,max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.max_features = max_features  # if None, use sqrt(n_features) for classification
        self.trees = []

    def bootstrap_sample(self,X,y):
        nsamples,nfeatures=X.shape
        idx=np.random.choice(nsamples,size=nsamples,replace=True)
        return X[idx],y[idx]

    def feature_subset(self,X):
        nsamples,nfeatures=X.shape
        
        if self.max_features is None:
            feat=int(np.sqrt(nfeatures))
        else:
            feat=self.max_features
        features_idx=np.random.choice(nfeatures,size=feat,replace=False)
        return features_idx

    def fit(self,X,y):
        self.trees=[]
        for _ in range(self.n_estimators):
            tree=DecisionTreeClassifier(self.max_depth,self.min_samples)
            
            X_sample_size,y_sample_size=self.bootstrap_sample(X,y)
            features_idx=self.feature_subset(X)
            
            tree.fit(X_sample_size[:,features_idx],y_sample_size)
            self.trees.append((tree,features_idx))

    def predict(self,X):
        tree_preds=[]
        for tree,features_idx in self.trees:
            pred=tree.predict(X[:,features_idx])
            tree_preds.append(pred)

        tree_preds = np.array(tree_preds)  # shape = (n_trees, n_samples)
        return np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=0, arr=tree_preds)
            
